In [ ]:
import os
import sys
import requests

import h5py
import matplotlib.pyplot as plt
import numpy as np
from convert_rrsg_data import convert_data

In [ ]:
# Download data from Zenodo
if not os.path.isdir('rrsg_data'):
    os.mkdir('rrsg_data')
    
zenodo_files=["rawdata_brain_radial_96proj_12ch.h5", "rawdata_spiral_ETH.h5"]

for f in zenodo_files:
    
    if not os.path.isfile("rrsg_data/%s"%f):
        print("Downloading %s"%f)
        res = requests.get('https://zenodo.org/record/3975887/files/%s'%f)
        with open('rrsg_data/%s'%f, 'wb') as h5f:
            h5f.write(res.content)

In [ ]:
# Convert to riesling format
f = 'rawdata_brain_radial_96proj_12ch.h5'
convert_data(input_fname='rrsg_data/%s' % f,
             output_fname='riesling_data/riesling_%s' % f,
             matrix=[256, 256, 1], voxel_size=[0.78, 0.78, 2])

f = 'rawdata_spiral_ETH.h5'
convert_data(input_fname='rrsg_data/%s' % f,
             output_fname='riesling_data/riesling_%s' % f,
             matrix=[220, 220, 1], voxel_size=[1, 1, 2])

In [ ]:
check = h5py.File('riesling_data/riesling_rawdata_brain_radial_96proj_12ch.h5')
# check = h5py.File('../tut.h5')
print(check.keys())
print(check['trajectory'].shape)
traj = np.array(check['trajectory'])
plt.scatter(traj[::16,::8,0], traj[::16,::8,1])
check.close()

In [ ]:
!riesling hdr riesling_data/riesling_rawdata_brain_radial_96proj_12ch.h5

In [ ]:
h5_in = 'riesling_data/riesling_rawdata_brain_radial_96proj_12ch.h5'
fov = 234
h5_out = 'riesling_recon/rrsg_challenge_brain'
!riesling-ox recon --fov={fov} --sdc=pipe --os 2.5 --out={h5_out} -v {h5_in}
imgh5 = h5py.File('riesling_recon/rrsg_challenge_brain-recon.h5')
img = imgh5['image'][:]
fig = plt.figure()
plt.imshow(abs(img[0,0,:,:]))

In [ ]:
h5_in = 'riesling_data/riesling_rawdata_spiral_ETH.h5'
h5_out = 'riesling_recon/rrsg_challenge_spiral'
!riesling-ox recon --sdc=pipe --out={h5_out} -v {h5_in}
imgh5 = h5py.File('riesling_recon/rrsg_challenge_spiral-recon.h5')
img = imgh5['image'][:]
fig = plt.figure()
plt.imshow(abs(img[0,0,:,:]))